In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# 特徴量データをロード
df = pd.read_csv('bybit_btcusdt_20210101-20230330_features.csv', index_col=0)

In [5]:

# ターゲット変数の作成
price_change = (df['Close'].shift(-1) / df['Close'] - 1) * 100
conditions = [
    (price_change >= 0.1),
    (price_change <= -0.1)
]
choices = ['up', 'down']
df['target'] = np.select(conditions, choices, default='neutral')

# 不要な列を削除
df = df.drop(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], axis=1).dropna()


In [6]:
# データセットをトレーニングセットとテストセットに分割
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# LightGBMモデルのトレーニング
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=1000, early_stopping_rounds=100)

# 予測と評価
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(pred) for pred in y_pred]
class_map = {'up': 0, 'neutral': 1, 'down': 2}
y_test_mapped = y_test.map(class_map).values

accuracy = accuracy_score(y_test_mapped, y_pred_class)
print(f"Accuracy: {accuracy}")

print("Classification report:")
print(classification_report(y_test_mapped, y_pred_class))

/home/nske/miniconda3/envs/jp38/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


ValueError: Series.dtypes must be int, float or bool